In [1]:
import pandas as pd
import matplotlib.pyplot as plt

# mongoexport --uri="mongodb+srv://<username>:<password>@<cluster>/sample_supplies" --collection="sales" --out=sales.json
df = pd.read_json("sales.json")
df

,_id,saleDate,items,storeLocation,customer,couponUsed,purchaseMethod
0,{'$oid': '5bd761dcae323e45a93ccfe8'},{'$date': '2015-03-23T21:06:49.506Z'},"[{'name': 'printer paper', 'tags': ['office', ...",Denver,"{'gender': 'M', 'age': 42, 'email': 'cauho@wit...",True,Online
1,{'$oid': '5bd761dcae323e45a93ccfe9'},{'$date': '2015-08-25T10:01:02.918Z'},"[{'name': 'envelopes', 'tags': ['stationary', ...",Seattle,"{'gender': 'M', 'age': 50, 'email': 'keecade@h...",False,Phone
2,{'$oid': '5bd761dcae323e45a93ccfea'},{'$date': '2017-06-22T09:54:14.185Z'},"[{'name': 'notepad', 'tags': ['office', 'writi...",Denver,"{'gender': 'M', 'age': 51, 'email': 'worbiduh@...",False,In store
3,{'$oid': '5bd761dcae323e45a93ccfeb'},{'$date': '2015-02-23T09:53:59.343Z'},"[{'name': 'binder', 'tags': ['school', 'genera...",Seattle,"{'gender': 'F', 'age': 45, 'email': 'vatires@t...",False,In store
4,{'$oid': '5bd761dcae323e45a93ccfec'},{'$date': '2017-12-03T18:39:48.253Z'},"[{'name': 'backpack', 'tags': ['school', 'trav...",London,"{'gender': 'M', 'age': 40, 'email': 'dotzu@ib....",False,In store
...,...,...,...,...,...,...,...
4995,{'$oid': '5bd761deae323e45a93ce36b'},{'$date': '2016-01-20T17:29:10.225Z'},"[{'name': 'pens', 'tags': ['writing', 'office'...",Austin,"{'gender': 'F', 'age': 45, 'email': 'du@rum.py...",False,In store
4996,{'$oid': '5bd761deae323e45a93ce36c'},{'$date': '2014-11-29T10:09:41.573Z'},"[{'name': 'pens', 'tags': ['writing', 'office'...",Denver,"{'gender': 'F', 'age': 47, 'email': 'dup@katje...",False,In store
4997,{'$oid': '5bd761deae323e45a93ce36d'},{'$date': '2017-12-25T07:02:51.283Z'},"[{'name': 'binder', 'tags': ['school', 'genera...",Denver,"{'gender': 'M', 'age': 29, 'email': 'losuhse@e...",False,In store
4998,{'$oid': '5bd761deae323e45a93ce36e'},{'$date': '2013-12-07T11:06:25.51Z'},"[{'name': 'notepad', 'tags': ['office', 'writi...",Denver,"{'gender': 'M', 'age': 59, 'email': 'hu@pehabu...",False,Online


# Preparation

In [2]:
# Get ID from Json
df['_id'] = df['_id'].str.get('$oid') 

# Get sale date in DateTime Type
df['saleDate'] = pd.to_datetime(df['saleDate'].str['$date'], format='ISO8601') 

# Extract customer into many collumn
df = df.join(pd.json_normalize(df['customer']).add_prefix('customer_')) 

# Extract items array to many row
df = df.explode('items').reset_index() 

# Extract item into many collumn
df = df.join(pd.json_normalize(df['items']).add_prefix('items_'))

# Convert itemsprice.$numberDecimal to number
df['items_price.$numberDecimal'] = pd.to_numeric(df['items_price.$numberDecimal'])
df

,index,_id,saleDate,items,storeLocation,customer,couponUsed,purchaseMethod,customer_gender,customer_age,customer_email,customer_satisfaction,items_name,items_tags,items_quantity,items_price.$numberDecimal
0,0,5bd761dcae323e45a93ccfe8,2015-03-23 21:06:49.506000+00:00,"{'name': 'printer paper', 'tags': ['office', '...",Denver,"{'gender': 'M', 'age': 42, 'email': 'cauho@wit...",True,Online,M,42,cauho@witwuta.sv,4,printer paper,"[office, stationary]",2,40.01
1,0,5bd761dcae323e45a93ccfe8,2015-03-23 21:06:49.506000+00:00,"{'name': 'notepad', 'tags': ['office', 'writin...",Denver,"{'gender': 'M', 'age': 42, 'email': 'cauho@wit...",True,Online,M,42,cauho@witwuta.sv,4,notepad,"[office, writing, school]",2,35.29
2,0,5bd761dcae323e45a93ccfe8,2015-03-23 21:06:49.506000+00:00,"{'name': 'pens', 'tags': ['writing', 'office',...",Denver,"{'gender': 'M', 'age': 42, 'email': 'cauho@wit...",True,Online,M,42,cauho@witwuta.sv,4,pens,"[writing, office, school, stationary]",5,56.12
3,0,5bd761dcae323e45a93ccfe8,2015-03-23 21:06:49.506000+00:00,"{'name': 'backpack', 'tags': ['school', 'trave...",Denver,"{'gender': 'M', 'age': 42, 'email': 'cauho@wit...",True,Online,M,42,cauho@witwuta.sv,4,backpack,"[school, travel, kids]",2,77.71
4,0,5bd761dcae323e45a93ccfe8,2015-03-23 21:06:49.506000+00:00,"{'name': 'notepad', 'tags': ['office', 'writin...",Denver,"{'gender': 'M', 'age': 42, 'email': 'cauho@wit...",True,Online,M,42,cauho@witwuta.sv,4,notepad,"[office, writing, school]",2,18.47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27433,4999,5bd761deae323e45a93ce36f,2014-08-18 06:25:49.739000+00:00,"{'name': 'envelopes', 'tags': ['stationary', '...",New York,"{'gender': 'M', 'age': 33, 'email': 'lihzodud@...",False,Online,M,33,lihzodud@voblep.ie,3,envelopes,"[stationary, office, general]",5,24.12
27434,4999,5bd761deae323e45a93ce36f,2014-08-18 06:25:49.739000+00:00,"{'name': 'pens', 'tags': ['writing', 'office',...",New York,"{'gender': 'M', 'age': 33, 'email': 'lihzodud@...",False,Online,M,33,lihzodud@voblep.ie,3,pens,"[writing, office, school, stationary]",4,27.10
27435,4999,5bd761deae323e45a93ce36f,2014-08-18 06:25:49.739000+00:00,"{'name': 'binder', 'tags': ['school', 'general...",New York,"{'gender': 'M', 'age': 33, 'email': 'lihzodud@...",False,Online,M,33,lihzodud@voblep.ie,3,binder,"[school, general, organization]",3,29.09
27436,4999,5bd761deae323e45a93ce36f,2014-08-18 06:25:49.739000+00:00,"{'name': 'laptop', 'tags': ['electronics', 'sc...",New York,"{'gender': 'M', 'age': 33, 'email': 'lihzodud@...",False,Online,M,33,lihzodud@voblep.ie,3,laptop,"[electronics, school, office]",2,598.79


# Processing

In [3]:
# Calculate total_sales in each purchase
df['sales'] = df['items_price.$numberDecimal'] * df['items_quantity']
df

,index,_id,saleDate,items,storeLocation,customer,couponUsed,purchaseMethod,customer_gender,customer_age,customer_email,customer_satisfaction,items_name,items_tags,items_quantity,items_price.$numberDecimal,sales
0,0,5bd761dcae323e45a93ccfe8,2015-03-23 21:06:49.506000+00:00,"{'name': 'printer paper', 'tags': ['office', '...",Denver,"{'gender': 'M', 'age': 42, 'email': 'cauho@wit...",True,Online,M,42,cauho@witwuta.sv,4,printer paper,"[office, stationary]",2,40.01,80.02
1,0,5bd761dcae323e45a93ccfe8,2015-03-23 21:06:49.506000+00:00,"{'name': 'notepad', 'tags': ['office', 'writin...",Denver,"{'gender': 'M', 'age': 42, 'email': 'cauho@wit...",True,Online,M,42,cauho@witwuta.sv,4,notepad,"[office, writing, school]",2,35.29,70.58
2,0,5bd761dcae323e45a93ccfe8,2015-03-23 21:06:49.506000+00:00,"{'name': 'pens', 'tags': ['writing', 'office',...",Denver,"{'gender': 'M', 'age': 42, 'email': 'cauho@wit...",True,Online,M,42,cauho@witwuta.sv,4,pens,"[writing, office, school, stationary]",5,56.12,280.60
3,0,5bd761dcae323e45a93ccfe8,2015-03-23 21:06:49.506000+00:00,"{'name': 'backpack', 'tags': ['school', 'trave...",Denver,"{'gender': 'M', 'age': 42, 'email': 'cauho@wit...",True,Online,M,42,cauho@witwuta.sv,4,backpack,"[school, travel, kids]",2,77.71,155.42
4,0,5bd761dcae323e45a93ccfe8,2015-03-23 21:06:49.506000+00:00,"{'name': 'notepad', 'tags': ['office', 'writin...",Denver,"{'gender': 'M', 'age': 42, 'email': 'cauho@wit...",True,Online,M,42,cauho@witwuta.sv,4,notepad,"[office, writing, school]",2,18.47,36.94
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27433,4999,5bd761deae323e45a93ce36f,2014-08-18 06:25:49.739000+00:00,"{'name': 'envelopes', 'tags': ['stationary', '...",New York,"{'gender': 'M', 'age': 33, 'email': 'lihzodud@...",False,Online,M,33,lihzodud@voblep.ie,3,envelopes,"[stationary, office, general]",5,24.12,120.60
27434,4999,5bd761deae323e45a93ce36f,2014-08-18 06:25:49.739000+00:00,"{'name': 'pens', 'tags': ['writing', 'office',...",New York,"{'gender': 'M', 'age': 33, 'email': 'lihzodud@...",False,Online,M,33,lihzodud@voblep.ie,3,pens,"[writing, office, school, stationary]",4,27.10,108.40
27435,4999,5bd761deae323e45a93ce36f,2014-08-18 06:25:49.739000+00:00,"{'name': 'binder', 'tags': ['school', 'general...",New York,"{'gender': 'M', 'age': 33, 'email': 'lihzodud@...",False,Online,M,33,lihzodud@voblep.ie,3,binder,"[school, general, organization]",3,29.09,87.27
27436,4999,5bd761deae323e45a93ce36f,2014-08-18 06:25:49.739000+00:00,"{'name': 'laptop', 'tags': ['electronics', 'sc...",New York,"{'gender': 'M', 'age': 33, 'email': 'lihzodud@...",False,Online,M,33,lihzodud@voblep.ie,3,laptop,"[electronics, school, office]",2,598.79,1197.58


# Top 10 items sales

In [4]:
df.groupby("items_name").sales.agg("sum").sort_values(ascending=False)

items_name
laptop           6775977.07
backpack          817374.10
pens              581843.27
binder            511644.57
notepad           463615.48
envelopes         376658.49
printer paper     367459.29
Name: sales, dtype: float64

# Top 3 sales in each store location

In [5]:
df.groupby(['storeLocation', 'items_name']).sales.agg("sum").groupby('storeLocation', group_keys=False).nlargest(3)


storeLocation  items_name
Austin         laptop        1018494.05
               backpack       115217.59
               pens            74622.52
Denver         laptop        1961659.25
               backpack       245679.34
               pens           185008.39
London         laptop        1073098.72
               backpack       144529.14
               pens            92602.30
New York       laptop         694613.32
               backpack        82563.14
               pens            64029.90
San Diego      laptop         462973.58
               backpack        50997.42
               pens            42134.55
Seattle        laptop        1565138.15
               backpack       178387.47
               pens           123445.61
Name: sales, dtype: float64

# Rank of store location

In [6]:
df.groupby(['storeLocation']).sales.agg("sum").sort_values(ascending=False)

storeLocation
Denver       2921009.92
Seattle      2255947.69
London       1583066.79
Austin       1445603.11
New York     1016059.59
San Diego     672885.17
Name: sales, dtype: float64

# Table show count of purchase method in each gender

In [7]:
gender_method = df.groupby(['customer_gender', 'purchaseMethod']).size().reset_index(name='count')

gender_method_table = gender_method.pivot(index='customer_gender', columns='purchaseMethod', values='count')

gender_method_table['Online/Phone'] = gender_method_table['Online'] + gender_method_table['Phone']

gender_method_table

purchaseMethod,In store,Online,Phone,Online/Phone
customer_gender,,,,
F,8062,4270,1499,5769
M,7571,4314,1722,6036


# Monthly Sales

In [8]:
df.groupby([df.saleDate.dt.month, df.saleDate.dt.strftime('%B')]).sales.agg("sum").rename_axis(['Month No.', 'Month'])

Month No.  Month    
1          January      918041.47
2          February     733884.21
3          March        867736.82
4          April        756559.46
5          May          878892.77
6          June         769416.01
7          July         847051.30
8          August       791577.12
9          September    758566.50
10         October      841007.09
11         November     860402.92
12         December     871436.60
Name: sales, dtype: float64